In [ ]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
credits = pd.read_csv('drive/MyDrive/Data/NewCredits.csv')
keywords = pd.read_csv('drive/MyDrive/Data/newKeywords.csv')

movies = pd.read_csv('drive/MyDrive/Data/MoviesMetadata.csv').\
                     drop(['imdb_id', 'poster_path', 'status', 'title'], axis=1).\
                     drop([19730, 29503, 35587])

In [ ]:
movies['id'] = movies['id'].astype('object')
df = movies.merge(keywords, on='id')
movies['id'] = movies['id'].astype('int64')
df=  movies.merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

# Deep Learning


In [ ]:
ratings_df = pd.read_csv('drive/MyDrive/Data/ratings_small.csv')

ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

ratings_df = ratings_df.merge(df[['id', 'original_title', 'name_genres', 'overview']], left_on='movieId',right_on='id', how='left')
ratings_df = ratings_df[~ratings_df['id'].isna()]
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

ratings_df.head()

,userId,movieId,rating,date,original_title,name_genres,overview
0,1,1371,2.5,2009-12-14 02:52:15,Rocky III,Drama,"Now the world champion, Rocky Balboa is living..."
1,1,1405,1.0,2009-12-14 02:53:23,Greed,"Drama, History",Greed is the classic 1924 silent film by Erich...
2,1,2105,4.0,2009-12-14 02:52:19,American Pie,"Comedy, Romance","At a high-school party, four friends find that..."
3,1,2193,2.0,2009-12-14 02:53:18,My Tutor,"Comedy, Drama, Romance",High school senior Bobby Chrystal fails his Fr...
4,1,2294,2.0,2009-12-14 02:51:48,Jay and Silent Bob Strike Back,Comedy,When Jay and Silent Bob learn that their comic...


In [ ]:
movies_df = df[['id', 'original_title']]
movies_df.rename(columns={'id':'movieId'}, inplace=True)
movies_df.head()

,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [ ]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

In [ ]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

Total Data: 42407


In [ ]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 29837
Unique users: 671


In [ ]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["userId"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["original_title"])

    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [ ]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 97s 3s/step - root_mean_squared_error: 1.6169 - factorized_top_k/top_1_categorical_accuracy: 3.1429e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0072 - factorized_top_k/top_10_categorical_accuracy: 0.0177 - factorized_top_k/top_50_categorical_accuracy: 0.0996 - factorized_top_k/top_100_categorical_accuracy: 0.1689 - loss: 6811.6263 - regularization_loss: 0.0000e+00 - total_loss: 6811.6263
Epoch 2/3
35/35 [==============================] - 92s 3s/step - root_mean_squared_error: 1.0189 - factorized_top_k/top_1_categorical_accuracy: 9.4286e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0196 - factorized_top_k/top_10_categorical_accuracy: 0.0421 - factorized_top_k/top_50_categorical_accuracy: 0.1905 - factorized_top_k/top_100_categorical_accuracy: 0.3126 - loss: 6456.8756 - regularization_loss: 0.0000e+00 - total_loss: 6456.8756
Epoch 3/3
35/35 [==============================] - 89s 3s/step - root_mean_squared_error: 0.9876 -

In [ ]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

8/8 [==============================] - 18s 2s/step - root_mean_squared_error: 1.1076 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0103 - factorized_top_k/top_50_categorical_accuracy: 0.0402 - factorized_top_k/top_100_categorical_accuracy: 0.0743 - loss: 5923.9399 - regularization_loss: 0.0000e+00 - total_loss: 5923.9399

Retrieval top-100 accuracy: 0.074
Ranking RMSE: 1.108


In [ ]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))

In [ ]:
predict_movie(123, 5)


Top 5 recommendations for user 123:

1. Un long dimanche de fiançailles
2. The Greatest Story Ever Told
3. Dog Day Afternoon
4. The Goddess
5. Dr. Jekyll and Mr. Hyde


In [ ]:
ratings_df[ratings_df['userId'] == '123']

,userId,movieId,rating,date,original_title,name_genres,overview
7901,123,233,4.0,2001-07-01 20:57:06,The Wanderers,Drama,The streets of the Bronx are owned by 60’s you...
7902,123,288,5.0,2001-07-01 19:32:47,High Noon,Western,High Noon is about a recently freed leader of ...
7903,123,407,5.0,2001-07-01 20:57:57,Kurz und schmerzlos,"Drama, Thriller",Three friends get caught in a life of major cr...
7904,123,968,3.0,2001-07-01 20:59:01,Dog Day Afternoon,"Crime, Drama, Thriller",A man robs a bank to pay for his lover's opera...
7905,123,1968,4.0,2001-07-01 19:30:36,Fools Rush In,"Drama, Comedy, Romance",Alex Whitman (Matthew Perry) is a designer fro...
7906,123,1976,4.0,2001-07-01 19:31:51,Jezebel,"Drama, Romance","In 1850s Louisiana, the willfulness of a tempe..."
7907,123,2003,4.0,2001-07-01 19:31:51,Anatomie de l'enfer,Drama,A man rescues a woman from a suicide attempt i...
7908,123,2428,5.0,2001-07-01 20:57:06,The Greatest Story Ever Told,"Drama, History",All-star epic retelling of Christ's life.
7909,123,2502,5.0,2001-07-01 20:59:01,The Bourne Supremacy,"Action, Drama, Thriller",When a CIA operation to purchase classified Ru...
7910,123,2699,2.0,2001-07-01 20:55:42,Titanic,"Action, Drama, Romance",A story of the romances of two couples upon th...


In [ ]:
import tensorflow as tf

# Check if GPU is available
if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available')

# Enable GPU growth to allocate memory on an as-needed basis
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

# Move calculations to the GPU
with tf.device('/GPU:0'):
    model.fit(cached_train, epochs=8)


GPU is not available
Epoch 1/8
35/35 [==============================] - 94s 3s/step - root_mean_squared_error: 1.4377 - factorized_top_k/top_1_categorical_accuracy: 2.2857e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0070 - factorized_top_k/top_10_categorical_accuracy: 0.0172 - factorized_top_k/top_50_categorical_accuracy: 0.0958 - factorized_top_k/top_100_categorical_accuracy: 0.1647 - loss: 6814.9637 - regularization_loss: 0.0000e+00 - total_loss: 6814.9637
Epoch 2/8
35/35 [==============================] - 92s 3s/step - root_mean_squared_error: 1.0413 - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0183 - factorized_top_k/top_10_categorical_accuracy: 0.0419 - factorized_top_k/top_50_categorical_accuracy: 0.1912 - factorized_top_k/top_100_categorical_accuracy: 0.3100 - loss: 6455.1718 - regularization_loss: 0.0000e+00 - total_loss: 6455.1718
Epoch 3/8
35/35 [==============================] - 90s 3s/step - root_mean_square

In [ ]:
predict_movie(123, 5)


Top 5 recommendations for user 123:

1. Dog Day Afternoon
2. Jezebel
3. The Greatest Story Ever Told
4. Un long dimanche de fiançailles
5. La révolution française
